In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install sacrebleu
!pip install --upgrade --no-cache-dir gdown
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  

In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate  # This replaces load_metric
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
import pandas as pd

In [ ]:
!gdown 1-nLLvtOF_92WxC0-Uo4wJ5gDeQ9moS_2

Downloading...
From: https://drive.google.com/uc?id=1-nLLvtOF_92WxC0-Uo4wJ5gDeQ9moS_2
To: /content/EN-FR.zip
100% 13.3M/13.3M [00:00<00:00, 58.9MB/s]


In [ ]:
!unzip /content/EN-FR.zip -d /content/data/

Archive:  /content/EN-FR.zip
  inflating: /content/data/dev.csv   
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  


In [ ]:
dataset = load_dataset("csv", data_files="/content/data/train.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 76251
    })
})

In [ ]:
dataset['train'][8]

{'en': 'The truth was that an idiotic ambition had alone impelled Camille to leave Vernon.',
 'fr': "La vérité était qu'une ambition bête avait seule poussé Camille au départ."}

In [ ]:
val_data = pd.read_csv('/content/data/dev.csv')
ds_val = Dataset.from_pandas(val_data)

test_data = pd.read_csv('/content/data/test.csv')
ds_test = Dataset.from_pandas(test_data)


dataset["validation"] = ds_val
dataset["test"] = ds_test

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 76251
    })
    validation: Dataset({
        features: ['en', 'fr'],
        num_rows: 25417
    })
    test: Dataset({
        features: ['en', 'fr'],
        num_rows: 25417
    })
})

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
tokenizer(['At length, however, the remarks of her companions on her absence of mind aroused her, and she felt the necessity of appearing more like herself.'])

{'input_ids': [[486, 2475, 6, 983, 6, 8, 21029, 13, 160, 9663, 7, 30, 160, 8605, 13, 809, 1584, 32, 10064, 160, 6, 11, 255, 1800, 8, 16696, 13, 16069, 72, 114, 6257, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Apply the preprocessing function over the entire dataset:
- use the ``map`` method.
- ``batched=True`` to process multiple elements of the dataset at once.

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/76251 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_data['train'][10])

{'en': 'But presently his whole attention was absorbed in twelve or fifteen pretty women who, seated opposite the dock, filled the three galleries above the bench and the jurybox.', 'fr': 'Mais bientôt toute son attention fut absorbée par douze ou quinze jolies femmes qui, placées vis-à-vis la sellette de l’accusé, remplissaient les trois balcons au-dessus des juges et des jurés.', 'input_ids': [13959, 1566, 12, 2379, 10, 299, 3, 25390, 112, 829, 1388, 47, 3, 19402, 16, 13369, 42, 17310, 1134, 887, 113, 6, 3, 22933, 6401, 8, 12908, 6, 3353, 8, 386, 18035, 756, 8, 8453, 11, 8, 12730, 2689, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3307, 21707, 2633, 520, 1388, 9620, 8074, 721, 260, 103, 10953, 407, 285, 29, 776, 13773, 15, 7, 9382, 285, 6, 6670, 1325, 4642, 18, 85, 18, 3466, 50, 1789, 1954, 20, 3, 40, 22, 6004, 302, 154, 6, 15636, 7, 7, 5635, 110, 5611, 19615, 7, 185, 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-c1372ad6b499>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # Removed tokenizer parameter as it's deprecated
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu
1,1.820900,1.608069,5.609100


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
trainer.save_model('T5_checkpoint')

In [ ]:
!cp -r /content/T5_checkpoint /content/drive/MyDrive/

cp: cannot create directory '/content/drive/MyDrive/': No such file or directory


In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."
text2 = "my name is John"
text3 = 'he died'

In [ ]:
def predict(sentence):
  inputs = tokenizer(sentence, return_tensors="pt").input_ids
  model = AutoModelForSeq2SeqLM.from_pretrained("/content/T5_checkpoint")
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
  outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return outputs

In [ ]:
pred = predict(text)
pred

'Les lumes partagent des ressources avec des bactéries fixatrices de l’azote.'